In [1]:
import pandas as pd

### 2021 source file spec

In [2]:
# Define the file path
file_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\raw\kdkf_2021_raw.xlsx'

# Load the Excel file to get sheet names
xls = pd.ExcelFile(file_path)

# Define the years
previous_year = 2020
start_year = 2021
target_year = 2022

# Set the pandas option to display floating point numbers using the decimal format
pd.options.display.float_format = '{:.5f}'.format

### Master Loop

In [3]:
# Initialize an empty DataFrame for the master data
master_2021 = pd.DataFrame()

# Loop over each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Check if the sheet name starts with "HRM2_KT_"
    if sheet_name.startswith("HRM2_KT_"):
        # Load the sheet into a DataFrame, skipping the first three rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=3)

        # Merge the first row with the column headers
        df.columns = df.iloc[0, :2].tolist() + (df.columns[2:]).tolist()
        df = df.iloc[1:].reset_index(drop=True)

        # Update Headers of the df
        rename_dict = {
            'HRM 2': 'HRM2-ID',
            'Unnamed: 2': 'HRM2-Bezeichnung',
            'Rechnung': f'Rechnung {previous_year}',
            'Budget*': f'Budget {start_year}',
            'Anpassung Budget**': f'Anpassung Budget {start_year}',
            'Budgetplus***': f'Budgetplus {start_year}',
            'Rechnung.1': f'Rechnung {start_year}',
            'Budget': f'Budget {target_year}'
        }
        df.rename(columns=rename_dict, inplace=True)

        # Filter the DataFrame to keep only rows where 'Referenz-ID' contains 'HRM2'
        df = df[df['Referenz-ID'].str.contains('HRM2', na=False)]

        # Add a new column with the DataFrame name (sheet name) and year spec
        df['Source'] = sheet_name
        df['Kanton'] = df['Source'].str.split('HRM2_KT_').str[1]
        df['Year'] = 2021

        # Use pd.concat to add the DataFrame to the master DataFrame
        master_2021 = pd.concat([master_2021, df], ignore_index=True)

# Display the first few rows of the master DataFrame
master_2021.head()

,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2020,Budget 2021,Anpassung Budget 2021,Budgetplus 2021,Rechnung 2021,Budget 2022,Source,Kanton,Year,Compte,Adaptation Budget**,Compte.1
0,HRM2_ER0030,30,Personalaufwand,25054355.61869,24189899.19114,203324.71926,24393223.91040,26152620.36052,26630564.07622,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
1,HRM2_ER0031,31,Sach- und übriger Betriebsaufwand,9432382.42794,9603541.23479,407451.21928,10010992.45407,10287100.33023,10415842.28070,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
2,HRM2_ER0314,davon 314,baulicher und betrieblicher Unterhalt,1100302.72593,1116254.81200,46215.04361,1162469.85561,1138027.15383,1106749.61800,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
3,HRM2_ER0318,davon 3180,Wertberichtigungen auf Forderungen,55150.77083,41547.87000,992.16048,42540.03048,753.92456,51273.26600,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
4,HRM2_neu_ER0033,33,Abschreibungen VV,2874363.86922,2882024.76714,-824.90492,2881199.86222,2829988.73580,2898268.65011,HRM2_KT_alle,alle,2021,NaN,NaN,NaN


### Translating FR columns to DE

In [4]:
# # Display rows where the column 'Kanton' is 'HRM2_KT_VD'
# filtered_rows = master_2022[master_2022['Kanton'] == 'HRM2_KT_VD']

# # Display the filtered rows
# filtered_rows.head()

In [5]:
# 1. Update 'Rechnung 2020' based on 'Compte'
master_2021['Rechnung 2020'] = master_2021.apply(
    lambda row: row['Compte'] if pd.isna(row['Rechnung 2020']) and not pd.isna(row['Compte']) else row['Rechnung 2020'],
    axis=1
)

# 2. Update 'Anpassung Budget 2021' based on 'Adaptation Budget**'
master_2021['Anpassung Budget 2021'] = master_2021.apply(
    lambda row: row['Adaptation Budget**'] if pd.isna(row['Anpassung Budget 2021']) and not pd.isna(row['Adaptation Budget**']) else row['Anpassung Budget 2021'],
    axis=1
)

# 3. Update 'Rechnung 2021' based on 'Compte.1'
master_2021['Rechnung 2021'] = master_2021.apply(
    lambda row: row['Compte.1'] if pd.isna(row['Rechnung 2021']) and not pd.isna(row['Compte.1']) else row['Rechnung 2021'],
    axis=1
)

# Display the first few rows of the updated DataFrame
master_2021.head()


,Referenz-ID,HRM2-ID,HRM2-Bezeichnung,Rechnung 2020,Budget 2021,Anpassung Budget 2021,Budgetplus 2021,Rechnung 2021,Budget 2022,Source,Kanton,Year,Compte,Adaptation Budget**,Compte.1
0,HRM2_ER0030,30,Personalaufwand,25054355.61869,24189899.19114,203324.71926,24393223.91040,26152620.36052,26630564.07622,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
1,HRM2_ER0031,31,Sach- und übriger Betriebsaufwand,9432382.42794,9603541.23479,407451.21928,10010992.45407,10287100.33023,10415842.28070,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
2,HRM2_ER0314,davon 314,baulicher und betrieblicher Unterhalt,1100302.72593,1116254.81200,46215.04361,1162469.85561,1138027.15383,1106749.61800,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
3,HRM2_ER0318,davon 3180,Wertberichtigungen auf Forderungen,55150.77083,41547.87000,992.16048,42540.03048,753.92456,51273.26600,HRM2_KT_alle,alle,2021,NaN,NaN,NaN
4,HRM2_neu_ER0033,33,Abschreibungen VV,2874363.86922,2882024.76714,-824.90492,2881199.86222,2829988.73580,2898268.65011,HRM2_KT_alle,alle,2021,NaN,NaN,NaN


In [6]:
len(master_2021)

5319

In [7]:
# # Display rows where the column 'Kanton' is 'HRM2_KT_VD'
# filtered_rows = master_2021[master_2021['Kanton'] == 'HRM2_KT_VD']

# # Display the filtered rows
# filtered_rows.head()

### Save df to .csv

In [8]:
# Define the path where the CSV file will be saved
output_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\ready\kdkf_2021_data.csv'

# Save the DataFrame to a CSV file
master_2021.to_csv(output_path, index=False)
